In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'C:\\Users\\maken\\y2015.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

C:\Users\maken\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas and see what happens off of this kind of naive approach.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

TypeError: '>' not supported between instances of 'str' and 'float'

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [3]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
2
term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
31
next_pymnt_d
4
last_credit_pull_d
32
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
8
hardship_status
3
hardship_start_date
6
hardship_end_date
7
payment_plan_start_date
7
hardship_loan_status
4


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [4]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [10]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
42537,NaN,NaN,5000.0,5000.0,0.0,36 months,7.75,156.11,A,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [12]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Sep-09,last_credit_pull_d_Sep-10,last_credit_pull_d_Sep-11,last_credit_pull_d_Sep-12,last_credit_pull_d_Sep-13,last_credit_pull_d_Sep-14,last_credit_pull_d_Sep-15,last_credit_pull_d_Sep-16,application_type_INDIVIDUAL,hardship_flag_N
0,NaN,NaN,5000.0,5000.0,4975.00000,10.65,162.87,24000.00,27.65,0.0,...,0,0,0,0,0,0,0,0,1,1
1,NaN,NaN,2500.0,2500.0,2500.00000,15.27,59.83,30000.00,1.00,0.0,...,0,0,0,0,0,0,0,0,1,1
2,NaN,NaN,2400.0,2400.0,2400.00000,15.96,84.33,12252.00,8.72,0.0,...,0,0,0,0,0,0,0,0,1,1
3,NaN,NaN,10000.0,10000.0,10000.00000,13.49,339.31,49200.00,20.00,0.0,...,0,0,0,0,0,0,0,0,1,1
4,NaN,NaN,3000.0,3000.0,3000.00000,12.69,67.79,80000.00,17.94,0.0,...,0,0,0,0,0,0,0,0,1,1
5,NaN,NaN,5000.0,5000.0,5000.00000,7.90,156.46,36000.00,11.20,0.0,...,0,0,0,0,0,0,0,0,1,1
6,NaN,NaN,7000.0,7000.0,7000.00000,15.96,170.08,47004.00,23.51,0.0,...,0,0,0,0,0,0,0,1,1,1
7,NaN,NaN,3000.0,3000.0,3000.00000,18.64,109.43,48000.00,5.35,0.0,...,0,0,0,0,0,0,0,0,1,1
8,NaN,NaN,5600.0,5600.0,5600.00000,21.28,152.39,40000.00,5.55,0.0,...,0,0,0,0,0,0,0,0,1,1
9,NaN,NaN,5375.0,5375.0,5350.00000,12.69,121.45,15000.00,18.08,0.0,...,0,0,0,0,0,0,0,0,1,1


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [5]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

#cross_val_score(rfc, X, Y, cv=10)

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matricies.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

Once you've taken a stab at this, review this [example solution]().

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'C:\\Users\\maken\\y2015.csv',
    skipinitialspace=True,
    header=1,
    skipfooter=2
)

# Note the warning about dtypes.

C:\Users\maken\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [5]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [11]:
# Your code here.

X = y2015[['loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','emp_length','home_ownership','annual_inc','verification_status','issue_d','pymnt_plan','purpose','title','dti','delinq_2yrs','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec','revol_bal','total_acc','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt']]
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

print(X)

        loan_amnt  funded_amnt  funded_amnt_inv  int_rate  installment  \
0           35000        35000          35000.0     11.99       778.38   
1           20000        20000          20000.0     12.88       672.73   
2            7200         7200           7200.0      9.17       229.53   
3           28000        28000          28000.0     14.85       663.92   
4           10000        10000          10000.0      7.91       312.95   
5           16000        16000          16000.0     13.44       367.67   
6           23975        23975          23975.0     19.89       633.73   
7           10000        10000          10000.0     11.99       222.40   
8           16250        16250          16250.0     13.44       373.41   
9           20200        20200          20200.0     18.49       518.35   
10          17600        17600          17600.0     19.89       465.22   
11          30000        30000          30000.0     15.77      1051.31   
12           1500         1500        

In [12]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

# Modify the X values to correspond to the kept features only.

rfc = ensemble.RandomForestClassifier()
#X = pca_fit #features to keep go here
X = pd.get_dummies(X)
X = X.dropna(axis=1)
Y = y2015['loan_status']

cross_val_score(rfc, X, Y, cv=10)

C:\Users\maken\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([ 0.98100394,  0.97665804,  0.97285809,  0.98123961,  0.97803268,
        0.97746271,  0.97979006,  0.97591907,  0.97862588,  0.97883962])

After going through the data and removing the columns that had a high number of NaN values or too many unique values, the rfc was still above 90%.